In [1]:
%config IPCompleter.greedy=True
import numpy as np
import cv2
import ModelCheck as mc
from Card import Card

In [2]:
def find_cards(og_frame, pp_frame):
    
    #List holding valid cards
    found_cards = []
    
    #Find all contours from given frame
    contours = cv2.findContours(pp_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    #Go through each contour as c
    for c in contours:
        
        size = cv2.contourArea(c)                       # areal
        peri = cv2.arcLength(c, True)                   # omkreds
        approx = cv2.approxPolyDP(c, 0.01*peri, True)   # reducere antal hjørner -> prøv at sæt den op til 0.05 (var 0.01)
    
        is_rectangle = len(approx) == 4                 # har 4 hjørner/punkter(vertices)
        is_reasonable_size = size>4000 and size<100000  # er firkanten en realistisk størrelse
    
    
        if(is_rectangle and is_reasonable_size):        # Prøv at lav figuren til et kort
            
            card = Card()
        
            card.contour = c
            
            card.coords = list(approx.reshape(4,2))
            
            card.width, card.height = cv2.boundingRect(c)[2:]
            
            card.set_center()
            
            card.find_warped_card(og_frame) # Find og extract kortet fra billedet og sæt det i self.warped 

            card.find_icons()
        
            if card.validate(): found_cards.append(card) # Hvis kortet er validt. Tilføj til found_cards
                
    return found_cards

def draw_predicted_hand(img, text):

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    TEXT = text
    FONT_SIZE = 2
    FONT_COLOR = (255, 0, 255)
    THICKNESS = 5

    ######

    # get boundary of this text
    text_w, text_h = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]
    
    img_h, img_w, _ = img.shape
    
    text_x = int(img_w/2 - text_w/2)
    text_y = int(img_h - text_h)

    cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)

    
def draw_warps_and_icons_from_cards(cards):
    
    # [Height,Width,Depth]
    s = [1080,1920,3]
    
    if(len(cards)) == 0: return np.zeros([540,1920,3], dtype=np.uint8)
    
    
    res1 = np.zeros((cards[0].warped_thresh.shape[0],0 ), dtype=np.uint8) #makes an empty 420 x 0 img to make the card in cards loop easier

    for card in cards[:5]:
        res1 = np.hstack((res1, card.warped_thresh))
    
    size_scale = s[0] / 2 / res1.shape[0] 
    res1 = cv2.resize(res1, (0,0), fx=size_scale, fy=size_scale )
    res1 = np.hstack((np.zeros((res1.shape[0], s[1]//2-res1.shape[1]), dtype=np.uint8), res1))
    res1 = cv2.cvtColor(res1, cv2.COLOR_GRAY2RGB)
        

    res2 = np.zeros((540,0,3), dtype=np.uint8)
    for card in cards[:5]:
        res2 = np.hstack((res2, cv2.resize(card.warped, (192,540))))
        
    res2 = np.hstack((res2,np.zeros((540, 960 - res2.shape[1], 3), dtype=np.uint8)))
    
    return np.hstack((res1,res2))
#     cv2.imshow("warps_thresh", res1)
#     cv2.imshow("warps", res2)


In [3]:

def preprocess_frame(frame):
    
    threshold = 160
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.threshold(blur,threshold,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_clahe(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    cl1 = clahe.apply(gray)
    thresh = cv2.threshold(cl1, 160 ,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_adap(frame):
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gaus = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 333, 1)
    return gaus



def find_cards_from_image(frame): # returns new frame & list of all cards found
    
    # prepross_frame = preprocess_frame(frame) 
    # prepross_frame = preprocess_frame_clahe(frame)
    prepross_frame = preprocess_frame_adap(frame)
    
    found_cards = find_cards(frame, prepross_frame) # returns a list of cards from the processed frame
    
    for card in found_cards:
        
        card.draw_card_outlines(frame) # outliner kortet
        card.draw_text(frame)          # tegner suit osv. over kortet


    predicted_hand = mc.getHand(found_cards)
    
    draw_predicted_hand(frame, predicted_hand)
    
    warps_img = draw_warps_and_icons_from_cards(found_cards)
    res = np.hstack((cv2.cvtColor(prepross_frame,cv2.COLOR_GRAY2RGB),frame))
    res = cv2.resize(res, (1920,int(1920*0.5625*0.5)))
    res = np.vstack((res,warps_img))
    res = cv2.resize(res, (0,0), fx=0.8, fy=0.8)
    
    return found_cards, res  

In [4]:
# def save_img_icon(img_path, save_path, suit=True, num=True):
    
#     img = cv2.imread(img_path)
    
#     prepross_frame = preprocess_frame(img) # processes frame to make it easier to find cards
    
#     found_cards = find_cards(img, prepross_frame) # returns a list of found cards
    
#     if(len(found_cards) == 0): return
    
#     card = found_cards[0] # take first card (there should only be one)
    
#     if suit: cv2.imwrite(save_path, card.suit_img)
#     if num: cv2.imwrite(save_path, card.num_img)

In [ ]:
def use_img(img):
    img = cv2.imread(img)
    
    cards, preview = find_cards_from_image(img)
    
#     cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     cv2.namedWindow("Feed2", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed2", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    cv2.imshow("Feed", img)
    cv2.imshow("Feed2", preview)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def use_cam(num=0):
    cap = cv2.VideoCapture(num)

    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'): break
        ret, frame = cap.read()
        cards, preview = find_cards_from_image(frame)
        
        cv2.imshow("Feed", frame)
        cv2.imshow("Feed2", preview)

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(0)

In [ ]:
use_img("imgs/img8.jpg")
# use_cam(1)
#save_img_icon("./pre/clubs.jpg", "./delete/this2.jpg", suit=False)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# num_imgs = {str(x):cv2.imread("./test_cards/"+str(x)+".jpg", cv2.IMREAD_GRAYSCALE) for x in range(1,14)}
# cv2.imshow("Feed", num_imgs[13])
# cv2.waitKey(0)
# cv2.destroyAllWindows()